In [7]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from scipy.io import wavfile
from Kildekode._07_Frekvensanalyse import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Frekvensanalyse med DFT

* **Emne IELEA2302 - Signalbehandling**
* **Uke 7, 2021**
* **Underviser: Kai Erik Hoff**

# Tema tirsdag 16. februar

* Repetisjon IDFT
* Repetisjon Signaloperasjner i Frekvensdomenet
* Spektral Lekkasje
* Vindusfunksjoner 
* Zero Padding

# Tema fredag 19. februar

* Repetisjon Spektral Lekkasje
* Repetisjon Vindusfunksjoner 
* Zero Padding
* Diskret Tids Fouriertransformasjon
* Vindusfunksjoner i Frekvensplanet
* Effektspekter

# Fouriertransformens "svakhet"

* Vi har så langt sett på Diskrét Fouriertransformasjon som en utgave av fourierrekke-dekomposisjon for digitale signal.
    * Så langt har alle eksempelsignalene vært signaler som kun er sammensatt av frekvenskomponenter som er periodiske over vinduslengden $N$. 
    * At et signal er periodisk over $N$ sampler betyr at det kun består av frekvenskomponenter med frekvens $\hat{\omega} = 2\pi\frac{k}{N}, \ \ \ k \in \mathbb{Z}$.
  
  
* Er dette en rimelig antagelse for signal fra den "virkelige verden"?

## Kodeeksempel 1:

* Generer $N=32$ sampler av det digitale signalet $x[n] = \cos\left(\frac{\pi}{10} \cdot n\right)$, og regn ut signalvinduets DFT-sekvens $X[k]$.
* Vis amplitudespekteret $\left| X[k]\right|$ som et stolpediagram.

In [4]:
N =32
n = np.arange(N)
xn = cos(pi/10*n)
Xk = np.fft.fft(xn)
plt.close(1);plt.figure(1)
plt.stem(n, np.abs(Xk))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<StemContainer object of 3 artists>

# Spektral lekkasje

* DFT av $N$ sampler fra et signal $x[n]$ vil i realiteten finne en måte å dekomponere akkurat disse $N$ samplene til $x[n]$ i $N$ frekvenskomponenter med digital frekens $\hat{\omega}_k = \Delta \hat{\omega}\cdot k = 2\pi \frac{k}{N}$.
* Dersom signalet $x[n]$ inneholder frekvenskomponenter med frekvens $\hat{\omega} \notin \hat{\omega}_k$, vil vi få spektral lekkasje.
    * Vi vil få utslag i en rekke frevkenser $\hat{\omega}_k$ som kompenserer for frekvenskomponenten som ikke er "på lista".
    * Størst utslag vil finne sted der $\hat{\omega}_k$ er nærmest $\hat{\omega}$.

* *Årsaken til at dette skjer blir tydelig når vi forsøker å rekonstruere den analoge bølgeformen til $x(t)$ med utgangspunkt i DFT-sekvensen $X[n]$.*

* Rekonstruksjon av analogt signal kan utføres ved å anse $X[k]$ som fourierrekke-koeffisienter $a_k=A_k\cdot e^{j\phi_k}$ for $0\leq k \leq \frac{N}{2}$, og generere et analogt signal med følgende formel:
$$x(t) = X[0] + 2\cdot \sum_{k=1}^{N/2} |X[k]| \cdot \cos\left(2\pi \frac{k\cdot f_s}{N}\cdot t + \angle X[k]\right)$$

## Signalrekonstruksjon illustrert 

* *Bruker for enkelhets skyld $f_s = N = 32$ for eksempelet*

<img src="Figurer/07_Frekvensanalyse/Fig1_SpecLeakTimeDomain.png" style="width: 80%; margin-left: 100px" />

# Refleksjoner
* En $N$-punkts DFT av et signal $x[n]$ gir frekvensinnholdet til et "hypotetisk" signal som ***er*** periodisk over vinduslengden på $N$ sampler uavhengig av om det relle signalet $x[n]$ er periodisk over dette sampleintervallet.
* Hvorvidt spektral lekkasje oppstår avhengig av hvor stor foskjell det er mellom det "hypotetiske" signalet og det reelle singalet.

## Regneeksempel 1:
* Et signal $x(t) = \cos(2\pi\cdot 570 \cdot t)$ samples med samplingsfrekvens $f_s = 4000Hz$. Et utsnitt med lengde $N=256$ sampler brukes deretter til å regne ut en DFT-sekvens $X[k]$ som vil være utgangspuntet til å utføre frekvensanalyse av signalet. For hvilken verdi $k$ finner vi elementet $X[k]$ med størst absoluttverid?

## Demo spektral lekkasje

In [8]:
SpectralLeakageDemo(figsize=(12,6)); # Figurstørrelsen kan endres for å tilpasse dokumentvisning.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Vindusfunksjoner

* En ***vindusfunksjon*** er et nyttig verktøy for å redusere konsekvensene av spektral lekkasje.
    * Med en spesiell manipulasjon av signalutklippet fra $x[n]$ *før* utregning av DFT, kan vi endre frekvensinnholdet i ønsket retning.
    * Vindusfunksjonen vil påføre en vekting av hver sample $x[n]$ avhengig av sampleverdien $n$.
    * Samplene nære "kantene" på vinduet vil krympes, slik at differansen mellom $x[0]$ og $x[N-1]$ blir minimal. 
    * Dette fjerner kunstige "sprang" i det hypotetiske periodiske signalet vi *faktisk* finner frekvensinnholdet til.

# Eksempel: hann vindu

### $$ w[n] = \begin{cases}
0.5 - 0.5 \cos \left( 2\pi \frac{n}{N-1} \right) \ \ ,& 0 \leq n <N \\
0, & \text{Otherwise}
\end{cases}$$

<img src="Figurer/07_Frekvensanalyse/Fig2_HannWindow.png" style="width: 80%; margin-left: 100px" />

## Kodeeksempel 2:
* Generer $N=32$ sampler av det digitale signalet $x[n] = \cos\left(\frac{\pi}{10} \cdot n\right)$, og regn ut signalvinduets DFT-sekvens $X[k]$.
* Utfør vekting av signalet $x[n]$ med en hann vindusfunksjon, regn ut DFT, og vis amplitudespekteret $\left| X[k]\right|$ som et stolpediagram.

In [ ]:
N =32
n = np.arange(N)
x_n = cos(pi/10*n)
w_n = 0.5 - 0.5*cos(2*pi * n /(N-1))
xw_n = x_n*w_n

Xw_k = np.fft.fft(xw_n)
plt.close(5); plt.figure(5)
plt.stem(n, np.abs(Xw_k), label='Med Hann vindu')
plt.stem(n, np.abs(np.fft.fft(x_n)), markerfmt='x', label='Uten vindusfunksjon')
plt.legend()

# Zero padding
* Metode for å få flere frekvenssampler uten å øke vinduslengden.
* "padder" signalet med nullsampler bak signalutklippet.
    * Dersom man bruker vindusfunksjon, gjøres dette *før* zero padding.
* Vi regner så ut DFT av det nye utvidede signalet.

<img src="Figurer/07_Frekvensanalyse/Fig3_ZeroPadding.jpg" style="width: 80%; margin-left: 100px" />

## Kodeeksempel 3:
1. Bruk zero padding til å øke doble antallet frekvenssampler DFT-sekvensen i forrige eksempel vi produsere.
2. Gjør justeringer slik at figuren viser tosidig frekvensspekter der x-aksen viser *Digital Frekvens*

# Diskret-tids Fouriertransformasjon (DTFT)

* Matematisk Transformasjon for å finne et ***kontinuerlig funksjonsuttrykk*** for frekvensinnholdet til et digitalt signal $x[n]$.
    * Forteller ***alt*** frekvensinnholdet til et signal i frekvensdomenet, ikke bare et endelig antall sampler.

<img src="Figurer/07_Frekvensanalyse/Fig4_DTFT1.png" style="width: 80%; margin-left: 100px" />

# Diskret-tids Fouriertransformasjon

* Formel:
### $$X\left(e^{j\hat{\omega}}\right) = \sum_{n=-\infty}^{\infty} x[n]\cdot e^{-j\hat{\omega}\cdot n}$$

* $X\left(e^{j\hat{\omega}}\right)$ er et *kontinuerlig* funksjonsuttrykk.
    * Regner ut frekvensinnholdet for et digitalt signal $x[n]$.
    * Forutsetter at signalet $x[n]$ kan beskrives som et matematisk funksjonsuttrykk.
    
    
* Samme grunnprinsipp som for FT, DFT; frekvensforskyv ved hjelp av frekvensmiksing, og regn ut areal/sum.

# Invers DTFT
* Formel:
$$ x[n] = \frac{1}{2\pi} \int_{-\pi}^{\pi}\left(e^{j\hat{\omega}}\right)\cdot e^{j\hat{\omega}\cdot n}$$

* Ettersom $X\left(e^{j\hat{\omega}}\right)$ er *kontinuerlig*, bruker vi her et *integral*.
    * Deler på $2\pi$ ettersom "frekvens-spennet" er $-\pi < \hat{\omega} < \pi$.



# Sentrale matematiske funksjonsuttrykk
* Vi må først definere et par elementere funksjonsuttrykk
    * Impulsfunksjon
    * Enhetsstegfunksjon

# Dirac's Deltapuls

<img src="Figurer/07_Frekvensanalyse/Fig5_deltapuls.png" style="width: 85%; margin-left: 50px" />

# Signal som funksjon av deltapulser

* Digitale signal kan skrives som en funksjon av deltapulser med ulik tidsforskyvning.
* Eksempel: $X[n] = \{1, 2, 3, -3, 0, 3\}$ kan besrkrives matematisk med følgende funksjon.
$$x[n] = \delta[n] + 2\delta[n-1] + 3\delta[n-2]-2\delta[n-3]+2\delta[n-5]$$

<img src="Figurer/07_Frekvensanalyse/Fig7_eksempelsignal.png" style="width: 80%; margin-left: 100px" />

# Enhetsstegfunksjon

<img src="Figurer/07_Frekvensanalyse/Fig6_stegfunksjon.png" style="width: 85%; margin-left: 50px" />

# Utsnintt av sinussignal
* Vi kan skrive et utsnitt med lengde $N$ av et sinussignal matematisk som punktvis multiplikasjon av sinussignalet med en ***rektangulær vindusfunksjon***.
<img src="Figurer/07_Frekvensanalyse/Fig8_Sinusutklipp.png" style="width: 100%" />

# Hva skjer med frekvensinnholdet?

* Vi vil vite hva frekvensinnholdet er for signalet $x_w[n]$ for *alle* frekvenser $-\pi <\hat{\omega} < \pi$. 
* Første steg er å finne frekvensinnholdet for de to komponentene $x[n]$ og $w_r[n]$ individuelt.

<img src="Figurer/07_Frekvensanalyse/Fig9_SineWinFourier.png" style="width: 85%; margin-left: 50px" />

# Sinussignal Frekvensinnhold

<img src="Figurer/07_Frekvensanalyse/Fig10_SineSpect.png" style="width: 85%; margin-left: 50px" />

# Rektangulært Vindu Frekvensinnhold
<img src="Figurer/07_Frekvensanalyse/Fig11_RectSpect.png" style="width: 85%; margin-left: 50px" />

## Regneeksempel 2: DTFT av rekgangulært vindu

* Regn ut $W_r\left(e^{j\hat{\omega}} \right)$ for den rektangulære vindusfunksjonen $w_r = u[n]-u[n-4]$.

## DTFT animasjon

In [ ]:
Video("Figurer/07_Frekvensanalyse/DTFT.mp4", embed=True, width=1000)

# DTFT av rektangulært vindu
* Rektangulært vindu har et generelt uttrykk for DTFT.

<img src="Figurer/07_Frekvensanalyse/Fig12_TransformPair.png" style="width: 85%; margin-left: 50px" />

# Tilbakeblikk: Amplitudemodulasjon

* Som vi utforsket i dataøving 1, vil multiplikasjon med en sinusbølge/sinussekvens resultere i *frekvensforskyvning*.
    * Matematisk formulering av denne relasjonen:
    
| Tidsdomene: $x[n]$ | Frekvensdomene: $X\left(e^{j\hat{\omega}}\right)$ |
|:-:|:-:|
| $$w[n]\cdot cos(\hat{\omega}_0\cdot n)$$ | $$\frac{1}{2}\left(W\left(e^{j(\hat{\omega}-\hat{\omega}_0}\right)+W\left(e^{j(\hat{\omega}+\hat{\omega}_0}\right)\right)$$|

<img src="Figurer/07_Frekvensanalyse/Fig13_WindowModulation.png" style="width: 100%" />

$$\cos(\hat{\omega}_0\cdot n) = \frac{1}{2}e^{j\hat{\omega}_0n} + \frac{1}{2}e^{-j\hat{\omega}_0n}$$

# DTFT av sinusbølge-utklipp
* DTFT av signalutsnittet $x_w[n] = \cos(\hat{\omega}_0\cdot n)\cdot w_r[n]$ vil være summen av to frekvensforskjøvne kopier av $W_r\left(e^{j\hat{\omega}}\right)$.

$$X_w\left(e^{j\hat{\omega}}\right) = \frac{1}{2}\left(W_r\left(e^{j(\hat{\omega}-\hat{\omega}_0}\right)+W_r\left(e^{j(\hat{\omega}+\hat{\omega}_0}\right)\right)$$

<img src="Figurer/07_Frekvensanalyse/Fig14_SineDTFT.png" style="width: 60%; margin-left: 100px" />

# DFT er sampling av DTFT
* Eksempel:
    16-punkts DFT tilsvarer å ta 16 sampler langs kurven til DTFT-kurven $X_w\left(e^{j\hat{\omega}}\right)$.
<img src="Figurer/07_Frekvensanalyse/Fig15_SineDFT.png" style="width: 60%; margin-left: 100px" />

# Hvordan motvirkes spektral lekkasje?

* Idealet for frekvensinnholdet av en vindusfunksjon $w_I[n]$ er en deltapuls $W_I\left(e^{j\hat{\omega}}\right) = \delta(\hat{\omega})$
<img src="Figurer/07_Frekvensanalyse/Fig16_IdealWindow.png" style="width: 60%; margin-left: 100px" />
* Dette er ikke mulig å oppnå uten et uendelig langt vindu.
    * Men vi kan finne noe som nærmer seg.

## Analsye av vindusfunksjoner

In [ ]:
from scipy.signal import get_window
def PlotWindow(N, window_name):
    DTFT_res = 1000
    n = np.arange(N)
    w_n = get_window(window=window_name, Nx=N)
    
    plt.subplot(2,1,1)
    plt.stem(n, w_n, markerfmt='.')
    
    omega = np.linspace(-pi, pi, DTFT_res)
    W_k = np.fft.fft(w_n, DTFT_res)
    W_k = np.fft.fftshift(W_k)
    
    plt.subplot(2,1,2)
    plt.plot(omega, 20*np.log10(np.abs(W_k)))
    plt.xlim([-pi, pi])
    ymax = round((20*np.log10(sum(w_n))+10)/20)*20
    plt.ylim(ymax=ymax, ymin=ymax-100)
    plt.grid(True)

In [ ]:
plt.close(6); plt.figure(6, figsize=(10,8))
PlotWindow(32, 'hamming')

# Effektspekter

<img src="Figurer/07_Frekvensanalyse/Fig17_PSD1.png" style="width: 80%; margin-left=100px" />

# Signalenergi og effekt
<img src="Figurer/07_Frekvensanalyse/Fig18_PSD2.png" style="width: 80%; margin-left=100px" />

# Periodogram
<img src="Figurer/07_Frekvensanalyse/Fig19_PSD4.png" style="width: 80%; margin-left=100px" />

## Kodeeksempel: frekvensanalyse av lydsignal

* Regn ut effektspekteret til et utsnitt av signalet `orca.wav` som starter i $t=0.25s$.
    * Vi velger passende vinduslengde selv.

In [ ]:
fs, data = wavfile.read(r'Datafiler/04_Frekvensrepresentasjon/orca.wav') # les audiofilen 'orca.wav' og lagre til variablene 'fs' og 'data'
xn = data/0x8000
sig_len = len(xn)
n = np.arange(sig_len)
t = n/fs
plt.close(7); plt.figure(7, figsize=(10,3))
plt.plot(t, xn)